## Ananlysis of 1D reads

### 1) Recovery 1D reads with a Phred quality > 5
    Retrieve all 1D reads with q >= 5 (the 1D reads from a 1D2 run are not split into pass/fail groups)
### 2) Split reads per sample using barcodes to identify each sample
    Using a fuzzy regex allowing for 3 mismatches 
### 3) Mapping of the reads against the reference
    Direct mapping using the ont2d settings against the references SNP regions (51 nt)
### 4) Detemine the SNP genotype on position 26 in the mappings 


###1) Recovery of 1D reads

In [1]:
# Init
#
import os, glob
import regex
from multiprocessing import Pool

rawDataDir       = '/media/genomics/nanopore/run_data/20171219_nanopore_tri-allelic-1D2_basecalled_albacore-2.1.3'
projectDir       = '/media/genomics/nanopore/projects/tri-allelic_SNPs/20171220_nanopore_1d2_analysis/Github'
rawDataQCDir     = os.path.join(projectDir, 'raw_data_qc_albacore-2.1.3')
resultDir        = os.path.join(projectDir, 'one_d_read_analysis_q5_or_better')
oneDFastqFile    = os.path.join(rawDataDir, 'workspace', 'fastq_runid_d4e8a3e33e86f12065040abb8f96841c1613b6b3.fastq')
snpRefFile       = os.path.join(projectDir, 'triallelic_snp_regions.fasta')
barcodeFile      = os.path.join(projectDir, 'barcodes_tri-allelic.csv')
sampleMap        = {
  'NB07': '9948',
  'NB08': '9947',
  'NB09': '2800',
  'NB10': 'Gednap 50 Person C',
  'NB12': 'Gednap 51 Person C'   
}

nanofilt   = 'NanoFilt'
nanoplot   = 'NanoPlot'
bwa        = '/opt/tools/bwa-0.7.15'
samtools   = '/opt/tools/samtools-1.3.1'
bcftools   = '/opt/tools/bcftools-1.3.1'


In [ ]:
! {nanofilt} --version 

In [8]:
# Rescue 1D reads with q factor 5 or better
#
oneDFastqQ5File = os.path.join(resultDir, os.path.basename(oneDFastqFile).replace('.fastq', '_q5.fastq'))

! cat {oneDFastqFile} | {nanofilt} -q 5 --readtype 1D > {oneDFastqQ5File}

print('Done')

5Done


In [10]:
# Original 1d (all q)
r = ! wc -l {oneDFastqFile}
count = int(r[0].split(' ')[0]) // 4
print('Original 1D reads (all q):        {:>12d}'.format(count))

# Recovered 1d (q => 5)
r = ! wc -l {oneDFastqQ5File}
count = int(r[0].split(' ')[0]) // 4
print('Recovered 1D reads having q >= 5: {:>12d}'.format(count))

Original 1D reads (all q):             1119243
Recovered 1D reads having q >= 5:       806463


In [13]:
# Generate quality score plots

# Rescued reads
! {nanoplot} -t 10 --maxlength 3000 --fastq_rich {oneDFastqQ5File} -o {rawDataQCDir} -p 'recovered_q5_one_d_'

print('Done')

Done


### 2) Split reads per sample using barcodes

In [17]:
# Utility functions
#
def reverseComplement(seq):
  transTab = str.maketrans('agctyrwskmdvhbAGCTYRWSKMDVHB', 'tcgarywsmkhbdvTCGARYWSMKHBDV')
  return seq.translate(transTab)[::-1]



def loadReads(fileName):
  """
  Return a dict with the sequencing reads (including quality scores) extracted from the given file
  """
  rData = {}
  with open(fileName, 'rt') as f:
    cnt = 0

    for line in f:
      cnt += 1
      if cnt % 4 == 1:
        readName = line.rstrip()
      elif cnt % 4 == 2:
        readSeq = line.rstrip()
      elif cnt % 4 == 0:
        readQual = line.rstrip()
        rData[readName] = {'s': readSeq, 'q': readQual}
        if len(rData[readName]['s'])==0 or len(rData[readName]['q'])==0:
          print('*** Partial read data: {}'.format(readName))
            
  return(rData)



def findBarcodes(readName):
  """
  Lookup all barcodes in the given read. Return a dict with the barcode hit counts for all reads.
  """
  barcodeHits = {}
  
  for barcodeName in barcodeList:
    # Lookup forward barcode sequence
    for pattern in (barcodeRE[barcodeName]['f'], barcodeRE[barcodeName]['r']):
      for match in pattern.finditer(readData[readName]['s']):
        if readName not in barcodeHits:
          barcodeHits[readName] = {}
        if barcodeName not in barcodeHits[readName]:
          barcodeHits[readName][barcodeName] = 1
        else:
          barcodeHits[readName][barcodeName] += 1
          
  return barcodeHits


In [18]:
# Load barcode data, compile regex
#
# Get the barcodes
#
barcodeList = {}
barcodeRE   = {}
maxMisMatch = 3

with open(barcodeFile, 'rt') as f:
  for line in f:
    line = line.strip()
    
    # Ignore the column header line (should start with a '#')
    if line.startswith('#'):
      continue
      
    # Store
    name, seq         = line.split(',')
    barcodeList[name] = {'f': seq, 'r': reverseComplement(seq)}
    
    barcodeRE[name] = {
      'f': regex.compile('(?e)({}){{e<={}}}'.format(barcodeList[name]['f'], maxMisMatch)),
      'r': regex.compile('(?e)({}){{e<={}}}'.format(barcodeList[name]['r'], maxMisMatch))
    }
    
print('Found {} barcodes:'.format(len(barcodeList)))
for n in sorted(barcodeList):
  print(n, barcodeList[n])

Found 5 barcodes:
NB07 {'f': 'GTGTTACCGTGGGAATGAATCCTT', 'r': 'AAGGATTCATTCCCACGGTAACAC'}
NB08 {'f': 'TTCAGGGAACAAACCAAGTTACGT', 'r': 'ACGTAACTTGGTTTGTTCCCTGAA'}
NB09 {'f': 'AACTAGGCACAGCGAGTCTTGGTT', 'r': 'AACCAAGACTCGCTGTGCCTAGTT'}
NB10 {'f': 'AAGCGTTGAAACCTTTGTCCTCTC', 'r': 'GAGAGGACAAAGGTTTCAACGCTT'}
NB12 {'f': 'CAGGTAGAAAGAAGCAGAATCGGA', 'r': 'TCCGATTCTGCTTCTTTCTACCTG'}


In [19]:
# Load 1D read data
#
readDataOneDQ5 = loadReads(oneDFastqQ5File)

print('Loaded {} 1D reads'.format(len(readDataOneDQ5)))

Loaded 804922 1D reads


In [20]:
# Identify barcodes in 1D reads
#
barcodeHitData1D  = {}
readData          = readDataOneDQ5
maxThread         = 20
pool              = Pool(maxThread)
bcHitList         = pool.map(findBarcodes, readDataOneDQ5.keys())  # list of {readname: {barcodename: hitcount}}
pool.terminate()

for d in bcHitList:
  barcodeHitData1D.update(d)

# Print stats
barcodeStats1D = {}
 
for readName in barcodeHitData1D:
  n = 'reads having {} barcodes'.format(len(barcodeHitData1D[readName]))
  if n in barcodeStats1D:
    barcodeStats1D[n] += 1
  else:
    barcodeStats1D[n] = 1
  
  for barcodeName in barcodeHitData1D[readName]:
    if barcodeName in barcodeStats1D:
      barcodeStats1D[barcodeName] += 1
    else:
      barcodeStats1D[barcodeName] = 1

for k in sorted(barcodeStats1D.keys()):
  print('{} = {} ({:>.1f}%)'.format(k, barcodeStats1D[k], 100*barcodeStats1D[k]/len(readDataOneDQ5)))
  
print()

NB07 = 85628 (10.6%)
NB08 = 102621 (12.7%)
NB09 = 211231 (26.2%)
NB10 = 182251 (22.6%)
NB12 = 79046 (9.8%)
reads having 1 barcodes = 596556 (74.1%)
reads having 2 barcodes = 30534 (3.8%)
reads having 3 barcodes = 1031 (0.1%)
reads having 4 barcodes = 15 (0.0%)



In [21]:
# Save 1D reads in fastq files per barcode (sample).  We keep only reads that have 1 type of barcode.
#
outFiles = {}

for readName in barcodeHitData1D:
  if len(barcodeHitData1D[readName]) == 1:
    for barcodeName in barcodeHitData1D[readName]:
      if barcodeName not in outFiles:
        fastqFileName = os.path.join(resultDir, '{}.fastq'.format(barcodeName))
        outFiles[barcodeName] = open(fastqFileName, 'wt')
        print('Created {}'.format(fastqFileName))
      
      # Write fastq read
      outFiles[barcodeName].write('{}\n{}\n+\n{}\n'.format(readName, readDataOneDQ5[readName]['s'], readDataOneDQ5[readName]['q']))
      
# Close files (required, otherwise buffers are not always flushed to disk!)
for barcodeName in outFiles:
  outFiles[barcodeName].close()
  
print('Done')

Created /media/genomics/nanopore/projects/tri-allelic_SNPs/20171220_nanopore_1d2_analysis/one_d_read_analysis_q5_or_better/NB09.fastq
Created /media/genomics/nanopore/projects/tri-allelic_SNPs/20171220_nanopore_1d2_analysis/one_d_read_analysis_q5_or_better/NB12.fastq
Created /media/genomics/nanopore/projects/tri-allelic_SNPs/20171220_nanopore_1d2_analysis/one_d_read_analysis_q5_or_better/NB10.fastq
Created /media/genomics/nanopore/projects/tri-allelic_SNPs/20171220_nanopore_1d2_analysis/one_d_read_analysis_q5_or_better/NB07.fastq
Created /media/genomics/nanopore/projects/tri-allelic_SNPs/20171220_nanopore_1d2_analysis/one_d_read_analysis_q5_or_better/NB08.fastq
Done


In [22]:
# Count 1D reads per barcode (sample)
#
barcodeList  = ['NB07', 'NB08', 'NB09', 'NB10', 'NB12']
barcodeReads = {}
totalReads   = len(readDataOneDQ5)

print('Total reads: {}'.format(totalReads))

for barcodeName in barcodeList:
  fastqFileName             = os.path.join(resultDir, '{}.fastq'.format(barcodeName))
  barcodeReads[barcodeName] = loadReads(fastqFileName)
  readCount                 = len(barcodeReads[barcodeName])
  
  print('{}: {} ({} %) reads'.format(barcodeName, readCount, 100*readCount/totalReads))

Total reads: 804922
NB07: 82158 (10.206951729484347 %) reads
NB08: 74436 (9.247604115678289 %) reads
NB09: 202455 (25.152126541453708 %) reads
NB10: 174928 (21.73229207302074 %) reads
NB12: 62579 (7.774542129547956 %) reads


### 3) Mapping of the reads against the reference

In [23]:
snpRefFile    = os.path.join(projectDir, 'triallelic_snp_regions.fasta')
sampleMap     = {
  'NB07': '9948',
  'NB08': '9947',
  'NB09': '2800',
  'NB10': 'Gednap 50 Person C',
  'NB12': 'Gednap 51 Person C'  
}

bwa        = '/opt/tools/bwa-0.7.15'
samtools   = '/opt/tools/samtools-1.3.1'
bcftools   = '/opt/tools/bcftools-1.3.1'

In [24]:
# Map amplicons to reference SNP region sequences (51 nt), call variants
#
for sampleName in sorted(sampleMap):
  # Map amplicons
  fastqFile = os.path.join(resultDir, '{}.fastq'.format(sampleName))
  fastaFile = os.path.join(resultDir, '{}_noqual.fasta'.format(sampleName))
  bamFile   = fastaFile.replace('.fasta', '_direct_mapping.bam')
  vcfFile   = fastaFile.replace('.fasta', '_direct_mapping.vcf')
  
  # Convert fastq to fasta
  ! paste - - - - < {fastqFile} | cut -f 1,2 | sed 's/^@/>/' | tr "\t" "\n" > {fastaFile}
  
  # Map
  ! {bwa} mem -t 10 -x ont2d {snpRefFile} {fastaFile} | {samtools} view -Sb - | {samtools} sort -o {bamFile} -
  ! {samtools} index {bamFile}
  
  # Variant calling
  !{samtools} mpileup -d 100000 -Buf {snpRefFile} -t AD {bamFile} | {bcftools} call -V indels -m - > {vcfFile}

print('Done')

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 82158 sequences (55608980 bp)...
[M::mem_process_seqs] Processed 82158 reads in 24.277 CPU sec, 2.535 real sec
[main] Version: 0.7.15-r1140
[main] CMD: /opt/tools/bwa-0.7.15 mem -t 10 -x ont2d /media/genomics/nanopore/projects/tri-allelic_SNPs/20171220_nanopore_1d2_analysis/triallelic_snp_regions.fasta /media/genomics/nanopore/projects/tri-allelic_SNPs/20171220_nanopore_1d2_analysis/one_d_read_analysis_q5_or_better/NB07_noqual.fasta
[main] Real time: 6.568 sec; CPU: 24.860 sec
[mpileup] 1 samples in 1 input files
Note: Neither --ploidy nor --ploidy-file given, assuming all sites are diploid
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 74436 sequences (41952178 bp)...
[M::mem_process_seqs] Processed 74436 reads in 25.193 CPU sec, 4.337 real sec
[main] Version: 0.7.15-r1140
[main] CMD: /opt/tools/bwa-0.7.15 mem -t 10 -x ont2d /media/genomics/nanopore/projects/tri-allelic_SNPs/20171220_nanopore_1d2_analys

### 4) Detemine the SNP genotype on position 26 in the mappings 

In [25]:
def genotype(vcfFile, snpPos):
  """
  Extract genotype data at given position from a given vcf file.
  """
  genotypeData = {}
  
  with open (vcfFile, 'rt') as inFile:
    for line in inFile:
      if line.startswith('#'): continue
      
      snpName, pos, id, ref, alt, qual, filt, info, form, formValues = line.rstrip().split()
      
      if int(pos) == snpPos:
        k        = form.split(':')
        v        = formValues.split(':')         
        formData = {}
        
        for i in range(len(k)):
          formData[k[i]] = v[i]
          
        infoData = {}
        for t in info.split(';'):
          k,v = t.split('=')
          infoData[k] = v
          
        alleles = ref
        if alt != '.':
          alleles += ''.join(alt.split(','))
          
        depths                = [int(d) for d in formData['AD'].split(',')]
        gt                    = [alleles[int(i)] for i in formData['GT'].split('/') ]
        genotypeData[snpName] = {'pos': pos, 'alleles': {'A': 0, 'G': 0, 'C': 0, 'T':0}, 'genotype': '/'.join(gt), 'depth': int(infoData['DP'])}
        
        for i in range(len(alleles)):
          genotypeData[snpName]['alleles'][alleles[i]] = depths[i]
        
  return genotypeData


In [26]:
# Get all sample genotypes
#
excludeSnp = [] 
genotypes  = {}

for sampleName in sorted(sampleMap, key=lambda k: sampleMap[k]):
  print("SNP's for sample {} ({})".format(sampleName, sampleMap[sampleName]))
  genotypes[sampleName] = {}
  vcfFile               = os.path.join(resultDir, '{}_noqual_direct_mapping.vcf'.format(sampleName))
  
  d = genotype(vcfFile, 26)
  
  for snpName in sorted(d):
    t = sum(d[snpName]['alleles'].values())
    
    # Likely genotype
    gg = {}
    vv = sorted([int(c) for c in d[snpName]['alleles'].values()], reverse=True)
    
    for g, c in d[snpName]['alleles'].items():
      if c not in gg:
        gg[c] = [g]
      else:
        gg[c].append(g)
    # Second allele must be at least 10% of main allele or it is ignored.
    # Third and fourth alleles are always ignored if present.
    #if len(vv) > 1 and vv[1] > vv[0]/10:
    if len(vv) > 1 and vv[1] > vv[0]/4:
      if len(gg[vv[0]]) > 1:
        aa = ''.join(gg[vv[0]])
      else:
        aa = gg[vv[0]][0] + gg[vv[1]][0]
    else:
      aa = gg[vv[0]][0] + gg[vv[0]][0]
    
    o = '  {:<14}  '.format(snpName)
    
    for g in sorted(d[snpName]['alleles']):
      o += '  {}:{:<5}'.format(g, d[snpName]['alleles'][g])
    for g in sorted(d[snpName]['alleles']):
      if t > 0:
        o += '  {}:{:>3}%'.format(g, round(100*d[snpName]['alleles'][g]/t))
      else:
        o += '  {}:{:>3}%'.format(g, '???')
    o += '    {:>5}/{:<5} reads'.format(t, d[snpName]['depth'])
    o += '    {}'.format(''.join(sorted(aa)))
    genotypes[sampleName][snpName] = ''.join(sorted(aa))
    print(o)
  
  print('')
  

SNP's for sample NB09 (2800)
  rs1008686         A:0      C:0      G:0      T:8342   A:  0%  C:  0%  G:  0%  T:100%     8342/9048  reads    TT
  rs1112534         A:0      C:3703   G:73     T:0      A:  0%  C: 98%  G:  2%  T:  0%     3776/3979  reads    CC
  rs17287498        A:3850   C:600    G:4568   T:0      A: 43%  C:  7%  G: 51%  T:  0%     9018/9178  reads    AG
  rs2032582         A:3665   C:0      G:818    T:3569   A: 46%  C:  0%  G: 10%  T: 44%     8052/8288  reads    AT
  rs2069945         A:0      C:648    G:1948   T:0      A:  0%  C: 25%  G: 75%  T:  0%     2596/2773  reads    CG
  rs2307223         A:6383   C:0      G:0      T:0      A:100%  C:  0%  G:  0%  T:  0%     6383/6847  reads    AA
  rs2853525         A:0      C:3310   G:0      T:5529   A:  0%  C: 37%  G:  0%  T: 63%     8839/9014  reads    CT
  rs3091244         A:0      C:3350   G:582    T:0      A:  0%  C: 85%  G: 15%  T:  0%     3932/4508  reads    CC
  rs34741930        A:531    C:3756   G:0      T:0      A: 

In [ ]:
# Genotype overview
#
h = 'SNP           '
snpNames = set()

sortedSampleNames = sorted(sampleMap, key=lambda k: sampleMap[k])

for sampleName in sortedSampleNames:
  h += ' {:>10}'.format(sampleMap[sampleName])
  
  for snpName in genotypes[sampleName]:
    snpNames.add(snpName)
print(h)

for snpName in sorted(snpNames):
  r = '{:<14}'.format(snpName)
  
  for sampleName in sortedSampleNames:
    if snpName in genotypes[sampleName]:
      aa = ''.join(sorted(genotypes[sampleName][snpName]))
    else:
      aa = ''
    r += ' {:>10}'.format(aa)
    
    
  print(r)
